In [1]:
import numpy as np
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.python.framework import ops
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# compute the xent loss 
def ent(p):
    if p>1 or p<0: return('Not Probability')
    elif p==0 or p==1: return (0) 
    else: return -(p*np.log(p)+(1-p)*np.log(1-p))
    
# Data: (x4,x10)= four possibilities: Prob(y=1, x4, x10)
# (1,1) -> p(y=1)=0.65
# (0,1) -> p(y=1)=0.25
# (1,0) -> p(y=1)=0.45
# (0,0) -> p(y=1)=0.10 

# The question is: does the machine has the capability to probe the details to this level? 
# If not capable of probing x10 dependence, then x10 is essentially a hidden variable 
#    whose influence needs to be summer over (marginalized), so we have
#    p(y=1,x4) = sum_x10 p(y=1,x4,x10) = (0.65+0.45)/2 = 0.55 
# If not capable of probing neither x4 or x10, we need to sum over both (as if both were hidden units)
#    so we have p(y=1)= sum_x10,x4 p(y=1,x4,x10) = (0.65+0.25+0.40+0.10)/4 = 0.35 

# Got both x10 and x4
plist = [0.65,0.25,0.45,0.10]
print ("B: Xent Loss ", sum(ent(p) for p in plist)/len(plist))

# Got only x4
plist = [0.55, 0.175]
print ("F: Xent Loss ", sum(ent(p) for p in plist)/len(plist))

# Got none
plist = [0.35]
print ("N: Xent Loss ", sum(ent(p) for p in plist)/len(plist))

B: Xent Loss  0.5557508926896193
F: Xent Loss  0.5759326267664994
N: Xent Loss  0.6474466390346325


In [3]:
tf.reset_default_graph()
np.random.seed(2)
tf.set_random_seed(2)

# Configuration on Global Variables
# 1. RNN Cell Configuration (normally num_neuron = state_size, but not necessary)
#num_neuron = 10

# 2. Recurrent Steps (Unfolding)
num_steps = 25 # number of truncated backprop steps ('n' in the discussion above)
pos_1=3
pos_2=40

# 3. Data Input/State/Output 
#seq_size = 50000000
seq_size = 200000000
state_size = 40

num_classes = 2

# 4. Training (Mini-Batch, LearningRate, Dropout, ...)
batch_size = 400
learning_rate = 0.3

In [4]:
def gen_batch(seq_size, batch_size, num_steps): 
    X = np.random.choice([0,1], p=[0.5,0.5], size=seq_size)   
    pos1=4
    # ------ Set the position here ! ---------------------
    pos2=20
    plist=[0.65, 0.45, 0.25, 0.10]
    p2=np.zeros((2, 2))
    p2[1,1]=plist[0]
    p2[1,0]=plist[1]
    p2[0,1]=plist[2]
    p2[0,0]=plist[3]

    X = np.random.choice([0,1], p = [0.5, 0.5], size = seq_size)
    Prob = p2[np.roll(X,pos1), np.roll(X,pos2)]
    Y = (np.random.rand(len(X)) < Prob).astype(int) 
    
    epoch_size = seq_size // (batch_size*num_steps)
    
    X.resize(batch_size, epoch_size, num_steps)
    Y.resize(batch_size, epoch_size, num_steps)
    
    for i in range(epoch_size):
        yield (X[:,i,:],Y[:,i,:])

#init_state = tf.zeros(tf.float32,[batch_size, state_size])
#rnn_inputs = tf.one_hot(x, num_classes)

#init_state = tf.zeros([batch_size, state_size])
#cell = tf.nn.rnn_cell.BasicRNNCell(state_size)
#rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, initial_state=init_state, dtype="float32")

#rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, x, dtype="float32")

In [5]:

x = tf.placeholder(tf.int32, [batch_size, num_steps], name='input_placeholder')
y = tf.placeholder(tf.int32, [batch_size, num_steps], name='labels_placeholder')
#init_state=tf.placeholder(tf.float32, [batch_size, state_size+state_size])
init_state = tf.zeros([batch_size, state_size*2])

rnn_inputs = tf.one_hot(x, num_classes)

#cell = tf.nn.rnn_cell.BasicRNNCell(state_size)
#rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, initial_state=init_state, dtype="float32")

cell = tf.nn.rnn_cell.LSTMCell(state_size, state_is_tuple=False)
rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, initial_state=init_state, dtype='float32')

print("x, y, init_state", x.get_shape(), y.get_shape(), init_state.get_shape())

print("run_outputs", rnn_outputs.get_shape())
print("final_state", final_state.get_shape())


with tf.variable_scope('softmax'):
    W = tf.get_variable('W', [state_size, num_classes])
    b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))

logits = tf.reshape(
            tf.matmul(tf.reshape(rnn_outputs, [-1, state_size]), W) + b,
            [batch_size, num_steps, num_classes])

losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
total_loss = tf.reduce_mean(losses)
train_step = tf.train.AdagradOptimizer(learning_rate).minimize(total_loss)


 

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
x, y, init_state (400, 25) (400, 25) (400, 80)
run_outputs (400, 25, 40)
final_state (400, 80)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
# Training the network 
def train_network(seq_size, num_steps, state_size, printout=10):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        training_losses = []
               
        training_loss = 0
        training_state = np.zeros([batch_size, state_size*2])
        
        step = 0

        for (X, Y) in gen_batch(seq_size, batch_size, num_steps):
            training_loss_, training_state, _ = \
                sess.run([total_loss, final_state, train_step],
                              feed_dict={x:X, y:Y, init_state:training_state})
            training_loss += training_loss_
            if step % printout == 0 and step > 0:
                print("Step & Loss: ", step, training_loss/printout)
                training_losses.append(training_loss/printout)
                training_loss = 0
            step +=1
                
    return training_losses

training_losses = train_network(seq_size,num_steps,state_size, printout=200)
print('Num of Unfolding (num_steps) ', num_steps)
print('Num of Neurons in RNN Cell (state_size) ', state_size)
print('Sequence Length(seq_size) ', seq_size)
print('Batch,Steps,Epoch ', batch_size, num_steps, seq_size//(batch_size*num_steps))
plt.ylim(0.45, 0.68)
plt.plot(training_losses)



Step & Loss:  200 0.6467878025770187
Step & Loss:  400 0.6354857847094536
Step & Loss:  600 0.633861452639103
Step & Loss:  800 0.6325184607505798
Step & Loss:  1000 0.6295275154709816
Step & Loss:  1200 0.6240663433074951
Step & Loss:  1400 0.6206122502684593
Step & Loss:  1600 0.614347128868103
Step & Loss:  1800 0.6068462041020394
Step & Loss:  2000 0.6015508890151977
Step & Loss:  2200 0.5919282990694046
Step & Loss:  2400 0.5746005138754845
Step & Loss:  2600 0.5744057410955429
Step & Loss:  2800 0.5743591180443763
Step & Loss:  3000 0.5740392872691155
Step & Loss:  3200 0.5731964427232742
Step & Loss:  3400 0.5734378260374069
Step & Loss:  3600 0.5730319094657897
Step & Loss:  3800 0.5737210628390312
Step & Loss:  4000 0.5734602302312851
Step & Loss:  4200 0.5725761610269546
Step & Loss:  4400 0.5726654013991356
Step & Loss:  4600 0.5716512781381607
Step & Loss:  4800 0.571621362566948
Step & Loss:  5000 0.572330881357193
Step & Loss:  5200 0.5715770941972732
Step & Loss:  5400 0